In [1]:
#mm

In [2]:
from mmilanutil import *

In [3]:
mmptmaxabs

<function mmilanutil.mmptmaxabs(x)>

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("d:/ai/myvocab/runs")

In [3]:
writer.flush()


<a href="https://colab.research.google.com/github/Taaniya/exploring-gpt2-language-model/blob/main/Visualizing_gpt2_token_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook visualizes word token embeddings of GPT2 on Tensorboard projector

In [6]:
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
#! pip install transformers

In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from tensorboard.plugins import projector

import re
import os
from tqdm import tqdm

In [2]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B0F24BC2B0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/config.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/generation_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B0F24E9120>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/generation_config.json


In [9]:
word_embeddings = model.transformer.wte.weight      # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

NameError: name 'model' is not defined

In [8]:
print(word_embeddings.shape)

print(position_embeddings.shape)

NameError: name 'word_embeddings' is not defined

In [3]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B0F24E8460>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/tokenizer_config.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/vocab.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B0F24BC220>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/vocab.json


In [ ]:
tokenizer.pretrained_vocab_files_map

In [8]:
tokenizer.vocab

{'ism': 1042,
 'ĠShadow': 8843,
 'Ġunlaw': 16148,
 'ĠDeborah': 36976,
 'ĠKirin': 40262,
 'meaning': 24815,
 'ĠCrypt': 15126,
 '________________________________________________________________': 27193,
 'ĠRyder': 40238,
 'head': 2256,
 'lah': 9271,
 'ĠSatanic': 41518,
 'Ġrifle': 11575,
 'Ġpaid': 3432,
 'ĠPuzzle': 23966,
 'Ġcoupons': 45972,
 'Andy': 35314,
 'ĠSafe': 19978,
 'Ġvigilance': 49202,
 'Ġminimal': 10926,
 'cue': 15509,
 'ĠMansion': 38560,
 'Ġcynicism': 49509,
 'Female': 27273,
 'Ġast': 6468,
 'ĠCt': 43166,
 'ATURE': 40086,
 'Ġliner': 35940,
 'friendly': 13120,
 'Ġdefiant': 36219,
 'Ġchang': 1488,
 'Double': 25628,
 'ĠKuwait': 27028,
 'Ġcycle': 6772,
 'guide': 41311,
 'Ġphylogen': 43422,
 'Ġmonths': 1933,
 'ĠKristen': 44467,
 'Ġobjectives': 15221,
 'Ġrestraining': 35078,
 'Ġcocaine': 15144,
 'winter': 40078,
 'acon': 7807,
 'Ġwrite': 3551,
 'Finding': 36276,
 'Ġstand': 1302,
 'adan': 29157,
 'Ġendorsed': 17441,
 'Ġ344': 43686,
 'Ġdarkened': 44261,
 'Ġgracious': 43210,
 'ĠYing': 

**Creating list of tokens in vocab sorted by their index in vocab**

In [5]:
vocab_list = sorted(tokenizer.vocab.items(), key=lambda x:x[1])

**Verify if the resulting list is sorted by the token indices.**

In [6]:
for k,v in tokenizer.vocab.items():
    if v < 10:
        print(k, v)

) 8
% 4
* 9
( 7
' 6
& 5
# 2
$ 3
" 1
! 0


In [7]:
vocab_list[:10]

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9)]

**Save the sorted token labels from vocab as metadata file**

In [14]:
# Just create the metadata file
log_dir='d:\ai'
log_dir='.'

with open(os.path.join(log_dir, 'metadata5.tsv'), "w") as f:
##with open(os.path.join(log_dir, 'metadata5.tsv'), "wb") as f:
    
    #aaa tqdm
    for word, idx in tqdm(vocab_list):
        line = str(word.encode(encoding='iso-8859-1', errors='replace'))
        line = str(word.encode(encoding='ascii', errors='replace'))
        line = str(word.encode(encoding='utf-8', errors='replace'))
        
        ###line = word 
        
        line = re.sub("^b'", "", line)
        line = re.sub('^b"', "", line)
        line = re.sub("'$", "", line)
        line = re.sub('"$', '', line)
        
        line = re.sub(r'^[?]', '', line)
        line = re.sub(r'^[\\]xc4\\xa0', '', line)
        
        

        f.write("{}\n".format(line))

  0%|                                 | 0/50257 [00:00<?, ?it/s]


TypeError: a bytes-like object is required, not 'str'

**Save the word embeddings**

In [46]:
writer.add_embedding(model.transformer.wte.weight.detach().numpy())

AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'

In [16]:
embeddings = tf.Variable(model.transformer.wte.weight.detach().numpy())
checkpoint = tf.train.Checkpoint(embedding=embeddings)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'd:/ai/logs/vocab/embedding.ckpt-1'

Finally set up tensorboard projector's configuration. This creates a configuration file with .pbtxt extension.

In [20]:
# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

Run tensorboard to visualize the embeddings. Use UMAP for faster and cleaner visualizations. Search a few keywords and find their nearest neighbours in the 3D space and in the drop down.

In [ ]:
writer.flush()


In [ ]:
#mm

%load_ext tensorboard
%tensorboard --logdir d:/ai

#### References

1. [Tensorboard embedding projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin)

2. https://towardsdatascience.com/how-to-visualize-text-embeddings-with-tensorboard-47e07e3a12fb

3. https://github.com/huggingface/transformers/issues/1458